<a href="https://colab.research.google.com/github/yanting04/getting-started/blob/master/test2(3_con%2Bpool).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np
 

 
transform=transforms.Compose([
    transforms.RandomHorizontalFlip(),   
    transforms.RandomGrayscale(),   
    transforms.ToTensor(),  
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])   
 

trainset=torchvision.datasets.CIFAR10('data',train=True,
                                      download=True,
                                      transform=transform)
testset=torchvision.datasets.CIFAR10('data',train=False,
                                     download=True,
                                     transform=transform)

trainloader=torch.utils.data.DataLoader(trainset,batch_size=16,
                                        shuffle=True,
                                        num_workers=2)   
testloader=torch.utils.data.DataLoader(testset,batch_size=16,
                                       shuffle=True,
                                        num_workers=2)
classes=('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')
 
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1=nn.Sequential(   
            nn.Conv2d(   
                in_channels=3,   
                out_channels=16,   
                kernel_size=5,   
                stride=1,   
                padding=2,   
            ),   #(3,32,32)-->(16,32,32)
            nn.ReLU(),   
            nn.MaxPool2d(kernel_size=2),  #(16,32,32)-->(16,16,16)
        )
        self.conv2=nn.Sequential(   
            nn.Conv2d(16,32,5,1,2),  # (16,16,16) -->(32,16,16)
            nn.ReLU(),
            nn.MaxPool2d(2),   # (32,16,16)-->(32,8,8)
        )
        self.conv3=nn.Sequential(
            nn.Conv2d(32,64,5,1,2),   #(32,8,8)-->(64,8,8)
            nn.ReLU(),
            nn.MaxPool2d(2),   #(64,8,8)-->(64,4,4)
        )
        self.out=nn.Linear(64*4*4,10)  
    def forward(self,x):
        x=self.conv1(x)
        x=self.conv2(x)
        x=self.conv3(x)
        x=x.view(-1,64*4*4,)   
        output=self.out(x)
        return output
net=Net()
criterion=nn.CrossEntropyLoss()   
optimizer=optim.Adam(net.parameters(),lr=0.001)   
if __name__=='__main__':
    correct = 0
    total = 0
    for epoch in range(15):   
        for data in trainloader:
            inputs,labels=data
            outputs=net(inputs)
            loss=criterion(outputs,labels)   
            predicted = torch.max(outputs, 1)[1].data.numpy()
            total += labels.size(0)
            correct += (predicted == labels.data.numpy()).sum()
            optimizer.zero_grad()   
            optimizer.step()   
    print('train loss: %d%%' % (100 * correct / total))
    correct = 0
    total = 0
    for i,data in enumerate(testloader):   
        images,labels=data
        outputs=net(images)
        predicted=torch.max(outputs,1)[1].data.numpy()
        total+=labels.size(0)
        correct+=(predicted==labels.data.numpy()).sum()
    print('test loss: %d%%' % (100 * correct / total))
 

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified
train loss: 74%
test loss: 73%


In [9]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))


Accuracy of plane : 72 %
Accuracy of   car : 82 %
Accuracy of  bird : 61 %
Accuracy of   cat : 61 %
Accuracy of  deer : 71 %
Accuracy of   dog : 64 %
Accuracy of  frog : 76 %
Accuracy of horse : 81 %
Accuracy of  ship : 87 %
Accuracy of truck : 82 %
